In [ ]:
!cp -r /kaggle/input/diffusers/diffusers /kaggle/working/

In [ ]:
%cd /kaggle/working/diffusers

In [ ]:
!pip install . --no-build-isolation

In [ ]:
%cd /kaggle/working/diffusers/examples/unconditional_image_generation

In [ ]:
!pip install -r requirements.txt

In [ ]:
from pathlib import Path

config_text = """
compute_environment: LOCAL_MACHINE
deepspeed_config: {}
distributed_type: NO
gpu_ids: "all"  # Notice the quotes here
machine_rank: 0
main_process_ip: null
main_process_port: null
main_training_function: main
megatron_lm_config: {}
mixed_precision: fp16
num_machines: 1
num_processes: 1
rdzv_backend: static
same_network: true
use_cpu: false
"""

config_path = Path("/root/.cache/huggingface/accelerate/default_config.yaml")
config_path.parent.mkdir(parents=True, exist_ok=True)
config_path.write_text(config_text)

print(f"✅ Config file created at: {config_path}")

In [ ]:
!accelerate env

In [ ]:
!ls /kaggle/working/diffusers/examples/unconditional_image_generation

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"✅ GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("❌ No GPU detected.")

In [ ]:
!accelerate launch /kaggle/working/diffusers/examples/unconditional_image_generation/train_unconditional.py \
    --train_data_dir /kaggle/input/wiki-dataset/wiki \
    --resolution=64 \
    --center_crop \
    --random_flip \
    --output_dir="ddpm-ema-wiki" \
    --train_batch_size=16 \
    --num_epochs=1 \
    --gradient_accumulation_steps=1 \
    --use_ema \
    --learning_rate=1e-4 \
    --lr_warmup_steps=500 \
    --mixed_precision=no 


In [ ]:
import torch
from diffusers import DDPMPipeline
from PIL import Image
import numpy as np
import os

# Define image name and path
image_name = "image_grid_5x5.png"
image_path = os.path.join("./", image_name)

# Load the trained model
model_dir = "./ddpm-ema-wiki"  # Path to your trained model
pipeline = DDPMPipeline.from_pretrained(model_dir)
pipeline.to("cuda" if torch.cuda.is_available() else "cpu")  # Make sure it uses the GPU if available

# Generate 25 images
num_images = 25
images = [pipeline().images[0] for _ in range(num_images)]

# Create a grid
grid_size = 5
width, height = images[0].size
grid_image = Image.new('RGB', (grid_size * width, grid_size * height))

for i, img in enumerate(images):
    row = i // grid_size
    col = i % grid_size
    grid_image.paste(img, (col * width, row * height))

# Save the image with an appropriate name
grid_image.save(image_path)
print(f"✅ Imagem salva em: {image_path}")

# Exibir a imagem
grid_image.show()
